In [1]:
from comet_ml import Experiment

In [2]:
import numpy as np 
import shap
import pickle
import json
import time
import json
import pandas as pd
import torch
import math
import os
import torch.nn as nn
import torch.nn.functional as F
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, roc_curve, auc
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.svm import SVC
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('final_df.csv')

In [4]:
df_test = df[df['season'] == 20192020]
df_test_R = df_test[df_test['season_type'] == 'R']
df_test_P = df_test[df_test['season_type'] == 'P']

In [5]:
rf = pickle.load(open("random_forest", 'rb'))
xgb = pickle.load(open("XGBoost_hmtuning_model_v2.pickle",'rb'))
log1 = pickle.load(open("LogisticRegression_distance", 'rb'))
log2 = pickle.load(open("LogisticRegression_angle", 'rb'))
log3 = pickle.load(open("LogisticRegression_distance+angle", 'rb'))

## Features for RF

In [6]:
old_feat = ['game_id', 'season', 'season_type', 'event_id', 'home_team',
       'away_team', 'home_team_side_1st_period', 'attacking_team',
       'attacking_player', 'goalie', 'period', 'period_time', 'goal_ind',
       'shot_ind', 'x_coordinates', 'y_coordinates', 'shot_type', 'empty_net',
       'strength', 'gwg', 'previous_event_x_coordinates',
       'previous_event_y_coordinates', 'previous_event_period_time',
       'previous_event_type', 'previous_event_period', 'previous_attacking_team',
       'attacking_team_side']

df_feat = df.iloc[:,:60].columns.tolist()

RF_feat = [feat for feat in df_feat if feat not in old_feat]

## Features for XGBoost

In [7]:
feat_xgb = ['game_seconds', 'period', 'x_coordinates', 'y_coordinates',
       'distance_from_net', 'angle_from_net', 'previous_event_type',
       'previous_event_x_coordinates', 'previous_event_y_coordinates',
       'time_since_last_event', 'distance_from_last_event', 'rebound',
       'change_in_angle', 'speed', 'time_since_powerplay_started', '5v5',
       '4v4', '3v3', '5v4', '5v3', '4v3', '4v5', '3v5', '3v4',
       'shot_type_Backhand', 'shot_type_Deflected', 'shot_type_Slap Shot',
       'shot_type_Snap Shot', 'shot_type_Tip-In', 'shot_type_Wrap-around',
       'shot_type_Wrist Shot']

def preprocess(df):
    le = LabelEncoder()
    df["previous_event_type"] = le.fit_transform(df["previous_event_type"])
    
    return df

## Assign the proper features to each model

In [8]:
X_rf = df_test_R[RF_feat]
X_rf_P = df_test_P[RF_feat]
X_xgb = preprocess(df_test_R[feat_xgb])
X_xgb_P = preprocess(df_test_P[feat_xgb])
X_lr_D = df_test_R['distance_from_net'].values.reshape(-1,1)
X_lr_D_P = df_test_P['distance_from_net'].values.reshape(-1,1)
X_lr_A = df_test_R['angle_from_net'].abs().values.reshape(-1,1)
X_lr_A_P = df_test_P['angle_from_net'].abs().values.reshape(-1,1)
X_lr_DA = df_test_R[['distance_from_net','angle_from_net']]
X_lr_DA_P = df_test_P[['distance_from_net','angle_from_net']]
y = df_test_R['goal_ind']
y_P = df_test_P['goal_ind']


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Prediction for regular season 

In [ ]:
preds_rf = rf.predict(X_rf)
preds_xgb = xgb.predict(X_xgb)
preds_lr_d = log1.predict(X_lr_D)
preds_lr_a = log2.predict(X_lr_A)
preds_lr_da = log3.predict(X_lr_DA)

X has feature names, but LogisticRegression was fitted without feature names


## Accuracy and ROC for logistic regression models

In [ ]:
accuracy_lrd = accuracy_score(y,preds_lr_d)
roc_auc_lrd =  roc_auc_score(y,preds_lr_d)
accuracy_lra = accuracy_score(y,preds_lr_a)
roc_auc_lra =  roc_auc_score(y,preds_lr_a)
accuracy_lrda = accuracy_score(y,preds_lr_da)
roc_auc_lrda =  roc_auc_score(y,preds_lr_da)

## Accuracy and ROC for Random Forest

In [ ]:
accuracy_rf = accuracy_score(y,preds_rf)
roc_auc_rf =  roc_auc_score(y,preds_rf)
report_rf = classification_report(y, preds_rf, output_dict=True)

## Accuracy and ROC for XGBoost

In [ ]:
accuracy_xgb = accuracy_score(y,preds_xgb)
roc_auc_xgb =  roc_auc_score(y,preds_xgb)
report_xgb = classification_report(y, preds_xgb, output_dict=True)

## Plots for regular season

## Predictions for playoffs

In [ ]:
preds_rf_p = rf.predict(X_rf_P)
preds_xgb_p = xgb.predict(X_xgb_P)
preds_lr_d_p = log1.predict(X_lr_D_P)
preds_lr_a_p = log2.predict(X_lr_A_P)
preds_lr_da_p = log3.predict(X_lr_DA_P)

AttributeError: 'XGBClassifier' object has no attribute 'prodict'

## Accuracy and ROC for logistic regression models - playoffs

In [ ]:
accuracy_lrd_p = accuracy_score(y_P,preds_lr_d_p)
roc_auc_lrd_p =  roc_auc_score(y_P,preds_lr_d_p)
accuracy_lra_p = accuracy_score(y_P,preds_lr_a_p)
roc_auc_lra_p =  roc_auc_score(y_P,preds_lr_a_p)
accuracy_lrda_p = accuracy_score(y_P,preds_lr_da_p)
roc_auc_lrda_p =  roc_auc_score(y_P,preds_lr_da_p)

## Accuracy and ROC for Random Forest - playoffs

In [ ]:
accuracy_rf_p = accuracy_score(y_P,preds_rf_p)
roc_auc_rf_p =  roc_auc_score(y_P,preds_rf_p)
report_rf_p = classification_report(y_P, preds_rf_p, output_dict=True)

## Accuracy and ROC for XGBoost - playoffs

In [ ]:
accuracy_xgb_p = accuracy_score(y_P,preds_xgb_p)
roc_auc_xgb_p =  roc_auc_score(y_P,preds_xgb_p)
report_xgb_p = classification_report(y_P, preds_xgb_p, output_dict=True)

## Plots for playoffs 